# Motion of a bell

In this notebook, we solve the equations of motion for a freely swinging bell, ignoring all other
effects such as the clapper.

In [25]:
%matplotlib inline
from scipy.special import ellipk
import scipy.integrate as integrate
import numpy as np
from functools import partial
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider, Layout

In [26]:
# acceleration due to gravity
g = 9.81

The equations of motion are derived in the appendix. Writing them as two coupled first order differential equations for the angle $\theta$ and angular velocity $v_\theta$:

$$\begin{align}
\dot{\theta} && = && v_\theta \\
\dot{v}_\theta && = && - \frac{g}{l_b} \sin \theta
\end{align}
$$

We can use the ODE solver from SciPy.  Although it turns out this isn't so accurate, if you set the bell swinging with large oscillations (very close to 180 degrees), then sometimes it swings more than 180 degrees at the next stroke (which means it keeps on swinging around in the same direction, since there isn't a stay!).  The RK4 solver used in the next section doesn't have this problem.

In [47]:
def deriv(l_b, y, t):
    [theta, vtheta] = y
    theta_dot  = vtheta
    vtheta_dot = - (g / l_b) * np.sin(theta)
    return [theta_dot, vtheta_dot]

Some functions to get the oscillation period. There is a well-known approximation for small oscillations
of a pendulum, which is independent of the amplitude (as long as the amplitude is quite small). We
also calculate an approximation that works for larger amplitudes; there are several such approximations
that have been proposed, here we use the Lima-Arun formula, from arXiv:physics/0510206v3.
Finally, the *exact* period can be calculated from the elliptic integral of the first kind.

In [48]:
# period of small oscillations
def small_oscillation(l_b):
    return 2 * np.pi * np.sqrt(l_b / g)

# Lima-Arun formula, from arXiv:physics/0510206v3
def lima_arun(l_b, theta_0):
    aa = np.cos(theta_0/2)
    return -small_oscillation(l_b) * np.log(aa) / (1 - aa)

# Exact formula from elliptic integral
def exact_period(l_b, theta_0):
    return 4 * np.sqrt(l_b / g) * ellipk(np.sin(theta_0/2)**2)

In [52]:
def update(theta_0, l_b):
    Texact = exact_period(l_b, theta_0 * np.pi/180)
    t = np.linspace(0, 2.5*Texact, 10000)
    y0 = [theta_0 * np.pi / 180, 0]
    y = integrate.odeint(partial(deriv, l_b), y0, t)
    fig, ax1 = plt.subplots(figsize=(10,8))
    ax1.set_xlabel('t')
    ax1.set_ylabel('angle', color='b')
    ax1.plot(t, y[:,0] * 180 / np.pi, 'b')
    ax2 = ax1.twinx()
    ax2.set_ylabel('velocity', color='g')
    ax2.plot(t, y[:,1] * 180 / np.pi, 'g')
    ax1.grid()
    ax2.grid()
    fig.tight_layout()
    plt.show()
    print('Period of small oscillations: {:.4} seconds'.format(small_oscillation(l_b)))
    print('Lima-Arun approximation for the period: {:.4}'.format(lima_arun(l_b, theta_0*np.pi/180)))
    print('Exact period: {:.4}'.format(Texact))

In [53]:
mylayout = Layout(width='50%', height='20px')
theta_0_float = FloatSlider(description=r'\(\theta_0\)', \
                        min=0.01, max=179.99, step=0.01, value=175, \
                        continuous_update=False, layout=mylayout)
l_b_float = FloatSlider(description=r'\(l_b\)', \
                        min=0.01, max=5, step=0.01, value=0.6, \
                        continuous_update=False, layout=mylayout)
interactive(update, theta_0=theta_0_float, l_b=l_b_float)

interactive(children=(FloatSlider(value=175.0, continuous_update=False, description='\\(\\theta_0\\)', layout=Layout(height='20px', width='50%'), max=179.99, min=0.01, step=0.01), FloatSlider(value=0.6, continuous_update=False, description='\\(l_b\\)', layout=Layout(height='20px', width='50%'), max=5.0, min=0.01, step=0.01), Output()), _dom_classes=('widget-interact',))

## Accelerometer

We now plot the signal as measured by an accelerometer if it is placed on the bell at an angle $\beta$ from the centre line of the bell, at a distance $r$ from the axis. We assume that the axes of the accelerometer are at angle $\delta$ relative to the direction of the axis of the bell. This means that if the accelerometer is placed such that the $Y$-axis is vertical, then we have $\delta = -\beta$.